In [1]:
import requests
from requests.auth import HTTPBasicAuth
import time
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta, timezone

correspond = ["icao24", "callsign", "origin_country", "time_position",
              "last_contact", "longitude", "latitude", "baro_altitude",
              "on_ground", "velocity", "true_track", "vertical_rate",
              "sensors", "geo_altitude", "squawk", "spi", "position_source", "category"]

# OpenSky credentials
USERNAME = "username"
PASSWORD = "password"

def send_opensky_to_kafka(topic, url, fields={}):
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',
    }

    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # response = requests.get(url, headers=headers, auth=HTTPBasicAuth(USERNAME, PASSWORD))
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data and "states" in data:
            for state in data["states"]:
                flight_info = {key: state[i] for i, key in enumerate(correspond) if key}
                flight_info["created_at"] = datetime.now(timezone.utc).isoformat()
                producer.send(topic, value=flight_info)
            producer.flush()    
            print(f"Sent {len(data['states'])} records to Kafka.")
        else:
            print("No valid data to send.")
    else:
        print(f"Failed to fetch data from OpenSky API. Status code: {response.status_code}")
        print(response.text)  # Print the response for debugging
    producer.close()

api_url_time = "https://opensky-network.org/api/states/all?extended={}&time={}"
api_url = "https://opensky-network.org/api/states/all?extended={}"



In [1]:
import time 
for i in range(10):
    for i in range(6):
        send_opensky_to_kafka("opensky", api_url.format(1), {})
        time.sleep(10)

  

NameError: name 'send_opensky_to_kafka' is not defined